In [1]:
from numpy import fliplr, flipud
from skimage import img_as_ubyte
import itertools
import skimage.io
# from losses_pytorch import dice_loss
# import hausdorff
import warnings
from albumentations import Compose, HorizontalFlip
import albumentations as A
from torch.nn import MSELoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from IPython.display import YouTubeVideo
from IPython.display import clear_output
from IPython.display import Image as show_gif
import skimage
from skimage import util
from skimage.util import montage
from skimage.transform import resize
import imageio
from scipy import stats
# import seaborn as sns
# import matplotlib.gridspec as gridspec
# import matplotlib.patches as mpatches
# import matplotlib.animation as anim
# from matplotlib import cm
import matplotlib.pyplot as plt
# import h5py
# import nilearn.plotting as nlplt
# import nilearn as nl
import pydicom as pdm
import nibabel as nib
# from sklearn.model_selection import KFold
# from sklearn.svm import SVR
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import train_test_split
import pandas as pd
# from scipy import stats
import numpy as np
from random import randint
import time
import os
# from tqdm import tqdm
import torch
# import torchvision.transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from GPUtil import showUtilization as gpu_usage
from numba import cuda
from my_metric import dice_coef_metric,jaccard_coef_metric
from util.diceloss import DiceLoss

/home/zhanghao/.conda/envs/ZH/lib/python3.9/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [8]:

from tkinter.messagebox import NO
from util.ccdataset3d import *
train_path, valid_path, test_path = getPathList()
train_loader = getDataloader_only1(
    train_path,valid_paths=None,test_paths=None, B1=1)
_,valid_loader,test_loader = getDataloader(
    train_path, valid_path, test_path,B1=1
)
len(train_loader),len(valid_loader),len(test_loader)
#(1,1,160,224,224)

(1, 43, 42)

In [3]:
KernelSize = 3
KernelSizePad = 1
SamePadding = 2
Stride = 2
lr = 0.00001


class Double_conv(nn.Module):
    #conv->bn->relu->conv->bn->relu
    def __init__(self, in_ch, out_ch):
        super(Double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, kernel_size=KernelSize, padding=1),
            nn.BatchNorm3d(out_ch),
            # nn.GroupNorm(16,out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, kernel_size=KernelSize, padding=1),
            nn.BatchNorm3d(out_ch),
            # nn.GroupNorm(16,out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.maxpool = nn.Sequential(
            nn.MaxPool3d(kernel_size=2, stride=Stride),
            Double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        return self.maxpool(x)


class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Up, self).__init__()
        self.up = nn.ConvTranspose3d(in_ch,in_ch,kernel_size=2,stride=2)
        self.upcv = nn.Conv3d(in_ch, out_ch, kernel_size=3, padding=1)
        self.conv = Double_conv(in_ch, out_ch)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.upcv(x1)
        # diffZ = torch.tensor([x2.size()[2] - x1.size()[2]])
        # diffY = torch.tensor([x2.size()[3] - x1.size()[3]])
        # diffX = torch.tensor([x2.size()[4] - x1.size()[4]])
        # x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
        #                 diffY // 2, diffY - diffY // 2,
        #                 diffZ // 2, diffZ - diffZ // 2, ])

        x = torch.cat([x2, x1], dim=1)  # (B,C,D,H,W)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv3d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class compVnet(nn.Module):
    def __init__(self, bilinear=True):
        super(compVnet, self).__init__()
        self.bilinear = bilinear
        self.inc = Double_conv(1, 16)
        self.down1 = Down(16, 32)
        self.down2 = Down(32, 64)
        self.down3 = Down(64, 128)
        self.down4 = Down(128, 256)
        self.up1 = Up(256, 128)
        self.up2 = Up(128, 64)
        self.up3 = Up(64, 32)
        self.up4 = Up(32, 16)
        self.outc = OutConv(16, 1)
        self.up1x = Up(256, 128)
        self.up2x = Up(128, 64)
        self.up3x = Up(64, 32)
        self.up4x = Up(32, 16)
        self.outcx = OutConv(16, 1)
        self.incz = Double_conv(1, 16)
        self.down1z = Down(16, 32)
        self.down2z = Down(32, 64)
        self.down3z = Down(64,128)
        self.down4z = Down(128, 256)
        self.up1z = Up(256, 128)
        self.up2z = Up(128, 64)
        self.up3z = Up(64, 32)
        self.up4z = Up(32, 16)
        self.outcz = OutConv(16, 1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        y = self.up1x(x5, x4)
        y = self.up2x(y, x3)
        y = self.up3x(y, x2)
        y = self.up4x(y, x1)
        y = self.outcx(y)
        # z = torch.cat([x, y], dim=1)
        z = torch.sigmoid(x)+torch.sigmoid(y)
        z1 = self.incz(z)
        z2 = self.down1z(z1)
        z3 = self.down2z(z2)
        z4 = self.down3z(z3)
        z5 = self.down4z(z4)
        z = self.up1z(z5, z4)
        z = self.up2z(z, z3)
        z = self.up3z(z, z2)
        z = self.up4z(z, z1)
        z = self.outcz(z)

        return x, y, z

In [2]:
KernelSize = 3
Pool_stride = 2

class Double_conv(nn.Module):
    #conv->bn->relu->conv->bn->relu
    def __init__(self, in_ch, out_ch):
        super(Double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, kernel_size=KernelSize, padding=1),
            nn.BatchNorm3d(out_ch),
            # nn.GroupNorm(32,out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, kernel_size=KernelSize, padding=1),
            nn.BatchNorm3d(out_ch),
            # nn.GroupNorm(32, out_ch),
            nn.ReLU(inplace=True),
            
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.maxpool = nn.Sequential(
            nn.MaxPool3d(kernel_size=2, stride=Pool_stride),
            Double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        return self.maxpool(x)


class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Up, self).__init__()
        self.up = nn.ConvTranspose3d(in_ch, in_ch, kernel_size=2, stride=2)
        self.upcv = nn.Conv3d(in_ch, out_ch, kernel_size=3, padding=1)
        self.conv = Double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.upcv(x1)
        # diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        # diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        # x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
        #                 diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)  # (B,C,H,W)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv3d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class Unet(nn.Module):
    def __init__(self):
        super(Unet, self).__init__()
        self.inc = Double_conv(1, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        self.down4 = Down(256, 512)
        self.up1 = Up(512, 256)
        self.up2 = Up(256, 128)
        self.up3 = Up(128, 64)
        self.up4 = Up(64, 32)
        self.outc = OutConv(32, 1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x



In [11]:
#全局权重初始化
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('conv3d') != -1:
        nn.init.kaiming_normal(m.weight)
        m.bias.data.zero()

#设置随机种子


def seed_everthing(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

In [ ]:
from tensorboardX import SummaryWriter
import torch.nn.functional as F

def training(epochs, model, trainLoader, validLoader):
    # torch.cuda.set_device(0)
    dice_list = []
    seed_everthing(0)  # 设置全局随机种子
    lr = 1e-4
    dice_score = 0.
    iou_score = 0.
    writer = SummaryWriter('/data/zhanghao/skull_project/mae-main/base_line/3dUnet_cc_only1')
    step = 0
    weights_init(model)  # 模型权重初始化
    Loss = DiceLoss()  # 实例化DiceLoss
    model.train()  # 设置model为训练模式

#____________________________________训练______________________________________________
    for epoch in range(epochs):
        # batch_num = 0  # 批次数
        optimizer = optim.Adam(model.parameters(), lr=lr)  # 每个epoch设置一次优化器
        model.train()  # 设置为训练模式
        optimizer.zero_grad()#优化器清零
        for data in trainLoader:  

            image = data["image"]  
            target = data["mask"]

            image = image.to(f'cuda:{model.device_ids[1,2,0]}')
            target = target.to(f'cuda:{model.device_ids[1,2,0]}')

            out1, out2, outx = model(image)
            loss1 = Loss(out1, target)
            loss2 = -Loss(out2, target)
            lossx = F.mse_loss(outx, image)
            loss = loss1+loss2+lossx
            loss.backward()
            optimizer.step()
            # print("step=",step)
            writer.add_scalar(tag="loss1", scalar_value=loss1, global_step=step)
            writer.add_scalar(tag="loss2", scalar_value=loss2, global_step=step)
            writer.add_scalar(tag="lossx", scalar_value=lossx, global_step=step)
            writer.add_scalar(tag="loss", scalar_value=loss, global_step=step)

            # print("step=",step)

            writer.add_scalar(tag="loss", scalar_value=loss, global_step=step)
            step += 1
        print("loss=", loss)

        optimizer.zero_grad()#在每轮训练后清零，给验证留出空间

# ___________________________________验证_______________________________________________
        #每4个epoch进行一次验证
        if (epoch%4) == 0:
            with torch.no_grad():  # 表示在验证的时候不需要进行梯度计算
                model.eval()  # 设置model为验证模式
                dice_score = 0.
                iou_score = 0.
                for data in validLoader:  # validLoader共60个nii文件,batch_size=1
                    valid_image = data["image"]  
                    valid_target = data["mask"]

                    valid_image = valid_image.cuda(2)
                    valid_target = valid_target.cuda(2)
                    _, out, _ = model(valid_image)
                    dice_score += dice_coef_metric(out.cpu(), valid_target.cpu())
                    iou_score += jaccard_coef_metric(out.cpu(), valid_target.cpu())
                    
                dice_score /= len(valid_loader)  # 算出平均dice_score
                iou_score /= len(valid_loader)


                print("epoch=", epoch,
                    "dice_score=", dice_score,
                    "iou_score=", iou_score)

                print("------------------------------------------------------------------")
                writer.add_scalar(tag="dice_scalar", scalar_value=dice_score, global_step=epoch)
                writer.add_scalar(tag="iou_scalar", scalar_value=iou_score, global_step=epoch)
                torch.save(model.state_dict(), '/data2/zhanghao/mae_project/3d_cc_only1/' +
                                    str(epoch) + 'Vnet' + str(dice_score) + '.pth')

        
            #早停
            if epoch > 20:
                min = 999
                for i in range(int(epoch/4 - 5) , int(epoch/4 - 1) ):
                    if dice_list[i] < min:
                        min = dice_list[i]
                if dice_score < min:
                    lr /= 10
                    print(lr)
                    if lr < 1e-6:
                        break

            dice_list.append(dice_score)#依次放入epoch为0 4 8 12 16 20 24....



In [ ]:
from torch import nn
print("len(train_loader)=", len(train_loader), "len(valid_loader)=", len(valid_loader))
model = compVnet()
model = nn.DataParallel(model,device_ids=[1,2,0]) 
model.to(f'cuda:{model.device_ids[1,2,0]}')
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
training(epochs=600, model=model, trainLoader=train_loader, validLoader=valid_loader)

# 测试

In [3]:
from util.ccdataset3d import *
train_paths, valid_paths, test_paths = getPathList()
train_loader, valid_loader, test_loader = getDataloader(
    train_paths, valid_paths, test_paths)
len(train_loader),len(valid_loader),len(test_loader)
# torch.Size([1, 1,160, 224, 224]) torch.FloatTensor

(274, 29, 56)

In [4]:
model = Unet()
model.load_state_dict(torch.load('/data/zhanghao/skull_project/mae-main/base_line/3dUnet_oasis_alldata/184Vnet0.9873532482555935.pth'))
model = model.cuda(2)


with torch.no_grad(): #表示在验证的时候不需要梯度计算
    a = torch.ones([1,1,160,224,224]).cuda(2)
    x = model(a)
    x.shape

In [5]:
m1_dice_list = []
m1_iou_list = []
with torch.no_grad(): #表示在验证的时候不需要梯度计算

    model.eval()
    dice_score = 0.
    iou_score = 0.

    for data in test_loader:
        test_image = data["image"].cuda(2)
        test_target = data["mask"].cuda(2)


        out = model(test_image) 
        x = dice_coef_metric(out.cpu(), test_target.cpu())
        y = jaccard_coef_metric(out.cpu(), test_target.cpu())
        m1_dice_list.append(x)
        m1_iou_list.append(y)
        dice_score += x
        iou_score += y
        print(x, y)
    
    dice_score /= len(test_loader)
    iou_score /= len(test_loader)
    
    print(
        "mean_dice=", dice_score,
        "mean_iou=", iou_score,
        "std_dice=",np.std(m1_dice_list, ddof=1),
        "std_iou=",np.std(m1_iou_list, ddof=1))

0.73127884 0.5763905
0.9260733 0.86232454
0.75344473 0.60442144
0.9262328 0.86260116
0.4180332 0.26424906
0.62848973 0.45824647
0.7115481 0.5522504
0.46707562 0.3046958
0.6546576 0.48661038
0.93505716 0.87803507
0.93083507 0.8706188
0.90488666 0.826295
0.92883426 0.8671247
0.75198483 0.6025446
0.7897502 0.6525514
0.8942437 0.80871683
0.90844536 0.8322491
0.84463584 0.731056
0.7221101 0.56508
0.69257957 0.5297298
0.8455498 0.73242646
0.92161447 0.8546243
0.64881396 0.48018107
0.69514394 0.5327361
0.5056458 0.3383708
0.5187912 0.35024852
0.55211073 0.3813211
0.86450446 0.7613455
0.94506985 0.89586014
0.91100216 0.83655095
0.8717171 0.77260506
0.7397221 0.58695155
0.6742994 0.5086363
0.93635124 0.88031995
0.93752784 0.88240224
0.9407002 0.8880396
0.65820736 0.49054328
0.807174 0.6766905
0.6068669 0.43561298
0.93081623 0.87058586
0.94196653 0.8902994
0.59832793 0.42686728
0.8311215 0.71104187
0.8511786 0.7409146
9.387344e-16 9.387344e-16
0.7299204 0.5747044
0.92017126 0.8521456
0.8828873 0

In [6]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
# --------------------------------------------------------
# References:
# DeiT: https://github.com/facebookresearch/deit
# BEiT: https://github.com/microsoft/unilm/tree/master/beit
# --------------------------------------------------------

import timm.optim.optim_factory as optim_factory
import argparse
import datetime
import json
import numpy as np
import os
import time
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter

import timm

# assert timm.__version__ == "0.3.2" # version check
from timm.models.layers import trunc_normal_
from timm.data.mixup import Mixup
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy

import util.lr_decay as lrd
import util.misc as misc
from util.datasets import build_dataset
from util.pos_embed import interpolate_pos_embed
from util.misc import NativeScalerWithGradNormCount as NativeScaler

import models_vit

from engine_finetune import train_one_epoch, evaluate
from ast import arg
import my_seg_vit
import util.lr_sched as lr_sched
import math
import sys
from my_metric import dice_coef_metric,jaccard_coef_metric
import seg3d_2dencoder 
import maeseg2d3d 

[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.


In [7]:
from email.policy import strict


encoder_str='mae_vit_medical_half_patch16'
# encoder_path='/data/zhanghao/skull_project/mae-main/pretrain_model/oasis_cc359_half_pretrain_out/checkpoint-399.pth'
encoder = seg3d_2dencoder.__dict__[encoder_str](lossforpatch = True)
# encoder.load_state_dict(torch.load(encoder_path),strict=False)
model_str='mae_vit3d_medical_768_512_extra_windowsize4'
# model_path='/data/zhanghao/skull_project/mae-main/base_line/seg_768_512_win4_b6_3d_cc359_010data_half_norm/checkpoint-296.pth'
model = maeseg2d3d.__dict__[model_str](encoder= encoder)
# model.load_state_dict(torch.load(model_path)['model'])
path = '/data/zhanghao/skull_project/mae-main/base_line/3d_cc359_010data_half_norm_800_extra_freeze5/260ViT0.9693019154459931.pth'
model.load_state_dict(torch.load(path))
model = model.cuda(2)

In [9]:
m2_dice_list = []
m2_iou_list = []
with torch.no_grad(): #表示在验证的时候不需要梯度计算

    model.eval()
    dice_score = 0.
    iou_score = 0.

    for data in test_loader:
        test_image = data["image"]
        test_target = data["mask"]

        test_image = test_image.to('cuda:2', non_blocking=True)
        test_target = test_target.to('cuda:2', non_blocking=True)

        _,out = model(test_image, test_target) #[N,196,256]
        out = model.unpatchify3D(out) #[N,1,224,224]

        
        x = dice_coef_metric(out.cpu(), test_target.cpu())
        y = jaccard_coef_metric(out.cpu(), test_target.cpu())
        m2_dice_list.append(x)
        m2_iou_list.append(y)
        dice_score += x
        iou_score += y
        print(x, y)
    
    dice_score /= len(test_loader)
    iou_score /= len(test_loader)
    
    print(
        "mean_dice=", dice_score,
        "mean_iou=", iou_score,
        "std_dice=",np.std(m2_dice_list, ddof=1),
        "std_iou=",np.std(m2_iou_list, ddof=1))

0.957 0.91754556
0.97585857 0.9528553
0.9706333 0.9429422
0.967074 0.93624705
0.97597593 0.9530791
0.9749041 0.951037
0.87564737 0.7788014
0.9776052 0.9561915
0.97144884 0.94448274
0.97417325 0.94964695
0.96714306 0.93637663
0.9755366 0.95224154
0.95660293 0.91681576
0.9804682 0.96168476
0.96876216 0.93941677
0.978965 0.95879674
0.96523035 0.9327973
0.96806765 0.93811154
0.97544897 0.9520745
0.9648248 0.9320401
0.96501064 0.93238705
0.9711234 0.9438677
0.9644741 0.9313857
0.94793075 0.9010156
0.9689322 0.9397367
0.9734569 0.9482864
0.9572758 0.9180527
0.9651229 0.9325966
0.9774281 0.9558526
0.9696873 0.94115824
0.9720174 0.94555825
0.97094434 0.94352955
0.9789587 0.9587847
0.9589038 0.92105204
0.9709563 0.94355214
0.9514971 0.90748155
0.96293795 0.9285249
0.9701751 0.9420777
0.9694127 0.940641
0.975069 0.95135087
0.96499664 0.9323608
0.96936566 0.9405525
mean_dice= 0.9665963578791845 mean_iou= 0.9357854695547194 std_dice= 0.016161382 std_iou= 0.028397165


In [10]:
stats.ttest_ind(m1_dice_list, m2_dice_list),stats.ttest_ind(m1_iou_list, m2_iou_list),

(Ttest_indResult(statistic=-0.29834338855925147, pvalue=0.7661957793870071),
 Ttest_indResult(statistic=-0.040624969820090666, pvalue=0.9676936290528062))

: 